In [1]:
import binance_utils as bu

key, secret = bu.get_credentials()
client = bu.get_client(key, secret)
tickers = bu.get_tickers(client)
print(len(tickers), tickers)


305 ['BNBBUSD', 'BTCBUSD', 'XRPBUSD', 'ETHBUSD', 'BCHABCBUSD', 'LTCBUSD', 'LINKBUSD', 'ETCBUSD', 'TRXBUSD', 'EOSBUSD', 'XLMBUSD', 'ADABUSD', 'BCHBUSD', 'QTUMBUSD', 'VETBUSD', 'EURBUSD', 'ICXBUSD', 'BTSBUSD', 'BNTBUSD', 'ATOMBUSD', 'DASHBUSD', 'NEOBUSD', 'WAVESBUSD', 'XTZBUSD', 'BATBUSD', 'ENJBUSD', 'NANOBUSD', 'ONTBUSD', 'RVNBUSD', 'STRATBUSD', 'AIONBUSD', 'ALGOBUSD', 'BTTBUSD', 'TOMOBUSD', 'XMRBUSD', 'ZECBUSD', 'DATABUSD', 'SOLBUSD', 'CTSIBUSD', 'ERDBUSD', 'HBARBUSD', 'MATICBUSD', 'WRXBUSD', 'ZILBUSD', 'KNCBUSD', 'REPBUSD', 'LRCBUSD', 'IQBUSD', 'GBPBUSD', 'DGBBUSD', 'COMPBUSD', 'BKRWBUSD', 'SXPBUSD', 'SNXBUSD', 'VTHOBUSD', 'DCRBUSD', 'STORJBUSD', 'IRISBUSD', 'MKRBUSD', 'DAIBUSD', 'RUNEBUSD', 'MANABUSD', 'DOGEBUSD', 'LENDBUSD', 'ZRXBUSD', 'AUDBUSD', 'FIOBUSD', 'AVABUSD', 'IOTABUSD', 'BALBUSD', 'YFIBUSD', 'BLZBUSD', 'KMDBUSD', 'JSTBUSD', 'SRMBUSD', 'ANTBUSD', 'CRVBUSD', 'SANDBUSD', 'OCEANBUSD', 'NMRBUSD', 'DOTBUSD', 'LUNABUSD', 'IDEXBUSD', 'RSRBUSD', 'PAXGBUSD', 'WNXMBUSD', 'TRBBUSD', '

In [ ]:
import os

if not os.path.exists(os.path.join(os.getcwd(), 'data')):
    os.mkdir(os.path.join(os.getcwd(), 'data'))
# Arbitrary list of tickers
training_tickers = ['BNBBUSD', 'ETHBUSD', 'DASHBUSD', 'DOGEBUSD', 'ADABUSD']

start_d = '2021-01-01'
end_d = '2022-01-01'
for ticker in training_tickers:
    kline_df = bu.get_historical_klines(client, ticker, start_d, end_d, interval=bu.Consts.INTERVAL_5MIN) # interval = 5min
    kline_df.to_csv(os.path.join(os.getcwd(), 'data', ticker+'_5min.csv'))

In [2]:
# extract features from dataframes into datasets
import pandas as pd
files = os.listdir(os.path.join(os.getcwd(), 'data'))

df = pd.read_csv(os.path.join(os.getcwd(), 'data', 'ADABUSD_5min.csv'))

In [ ]:
# install pytorch
!pip install torch==1.10.2+cu102 torchvision==0.11.3+cu102 torchaudio===0.10.2+cu102 -f https://download.pytorch.org/whl/cu102/torch_stable.html
!pip install pytorch-forecasting

In [1]:
import feature_extraction

train, test, scale = feature_extraction.feature_dfs_from_klines(os.path.join(os.getcwd(), 'data', 'ADABUSD_5min.csv'))
train.head()

,open,high,low,close,volume,qav,n_trades,tbbav,tbqav,next_return
1,0.446094,0.270949,0.534510,0.445746,0.017527,0.001552,0.003846,0.007839,0.000767,0.451397
2,0.440888,0.268847,0.536890,0.451397,0.019038,0.001678,0.005182,0.014998,0.001464,0.475875
3,0.445287,0.284949,0.566399,0.475875,0.006384,0.000565,0.001337,0.007656,0.000749,0.454527
4,0.466151,0.277660,0.562801,0.454527,0.003517,0.000311,0.001337,0.003998,0.000391,0.451764
5,0.453803,0.280984,0.551099,0.451764,0.003055,0.000270,0.001282,0.002056,0.000201,0.467696


In [2]:
feature_extraction.rescale_df(train, scale[0], scale[1]).head()

,open,high,low,close,volume,qav,n_trades,tbbav,tbqav,next_return
1,-0.004185,-0.004184,-0.007195,-0.006586,135592.4,24375.286004,141.0,25875.7,4650.278297,-0.003956
2,-0.006636,-0.004921,-0.006411,-0.003956,147283.8,26357.865564,190.0,49508.3,8871.062678,0.007439
3,-0.004565,0.000722,0.003310,0.007439,49386.5,8876.441155,49.0,25273.7,4541.883710,-0.002498
4,0.005257,-0.001832,0.002125,-0.002498,27208.2,4884.727909,49.0,13197.4,2368.477989,-0.003785
5,-0.000556,-0.000667,-0.001730,-0.003785,23634.9,4235.342296,47.0,6786.3,1216.193052,0.003632


472610.8486176335

In [28]:
next(iter(dataset))

({'x_cat': tensor([], size=(31, 0), dtype=torch.int64),
  'x_cont': tensor([], size=(31, 0)),
  'encoder_length': 30,
  'decoder_length': 1,
  'encoder_target': tensor([-3.9557e-03,  7.4393e-03, -2.4985e-03, -3.7849e-03,  3.6317e-03,
           2.2825e-03,  1.7218e-03,  2.2179e-04, -7.2066e-04,  3.2731e-03,
          -2.4330e-03,  7.1504e-03, -1.3209e-03, -2.9208e-03,  3.2057e-03,
           3.6362e-03,  2.4702e-03,  5.7496e-03,  5.6623e-03, -4.6018e-03,
           3.8072e-04,  5.4369e-05,  5.4366e-05,  3.1530e-03, -5.4192e-04,
          -2.7653e-03,  2.7186e-04, -1.2502e-03, -2.0681e-03,  4.8538e-03]),
  'encoder_time_idx_start': tensor(1),
  'groups': tensor([0]),
  'target_scale': array([0.00087837, 0.00338244], dtype=float32)},
 (tensor([0.0015]), None))